In [11]:
using Pkg; Pkg.activate(joinpath(Pkg.devdir(), "MLCourse"))
using CSV, DataFrames, Plots, OpenML

  Activating project at `~/.julia/dev/MLCourse`


# Raw Data Visualisation

Let's start looking at the first 50 rows of the raw training data:

In [12]:
train_data = CSV.read("DATA/train.csv", DataFrame, limit = 50)

,Xkr4,Gm1992,Gm19938,Gm37381,Rp1,Sox17,Gm37587,Gm37323,Mrpl15
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2.19038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.86198,0.0,0.506726,0.0,0.0,0.0,0.0,0.0,0.0
3,2.76676,0.0,0.629614,0.0,0.0,0.0,0.0,0.0,0.0
4,2.14643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.84005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.79552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.96702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3.02083,0.0,0.374098,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
describe(train_data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Xkr4,2.22804,0.0,2.19948,3.62105,0,Float64
2,Gm1992,0.0148774,0.0,0.0,0.432665,0,Float64
3,Gm19938,0.169924,0.0,0.0,1.53657,0,Float64
4,Gm37381,0.0241737,0.0,0.0,1.20868,0,Float64
5,Rp1,0.0,0.0,0.0,0.0,0,Float64
6,Sox17,0.0,0.0,0.0,0.0,0,Float64
7,Gm37587,0.0,0.0,0.0,0.0,0,Float64
8,Gm37323,0.0,0.0,0.0,0.0,0,Float64
9,Mrpl15,0.143806,0.0,0.0,1.45844,0,Float64


We can see that some genes have a mean expression of zero, plus a minimal value and a maximal value of 0. This type of parameters are useless to our model prediction. Therefore some data cleaning needs to be done. 

# Data Treatment

## 1-Removal of correlated parameters

In Data_treatment.jl :
- we tested if we had missing data: none found
- we removed  


In [15]:
train_data_treated = CSV.read("DATA/dataX.csv", DataFrame, limit = 50)

,Xkr4,Gm1992,Gm19938,Gm37381,Rp1,Sox17,Gm37323,Mrpl15,Lypla1
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2.19038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.86198,0.0,0.506726,0.0,0.0,0.0,0.0,0.0,0.0
3,2.76676,0.0,0.629614,0.0,0.0,0.0,0.0,0.0,0.0
4,2.14643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.84005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.79552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.96702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3.02083,0.0,0.374098,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
describe(train_data_treated)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,Xkr4,2.22804,0.0,2.19948,3.62105,0,Float64
2,Gm1992,0.0148774,0.0,0.0,0.432665,0,Float64
3,Gm19938,0.169924,0.0,0.0,1.53657,0,Float64
4,Gm37381,0.0241737,0.0,0.0,1.20868,0,Float64
5,Rp1,0.0,0.0,0.0,0.0,0,Float64
6,Sox17,0.0,0.0,0.0,0.0,0,Float64
7,Gm37323,0.0,0.0,0.0,0.0,0,Float64
8,Mrpl15,0.143806,0.0,0.0,1.45844,0,Float64
9,Lypla1,0.152868,0.0,0.0,1.45222,0,Float64


## 2- PCA